# Sample notebook for ARCS

This notebook demonstrates how to use the ARCS library to perform a simulation on a chemical reaction network. 

In [52]:
import pandas as pd
from arcs.traversal import traverse
from arcs.analysis import AnalyseSampling
import warnings

warnings.filterwarnings("ignore")

In [53]:
temperature = 300
pressure = 10
concs = {"SO2": 10e-6, "NO2": 50e-6, "H2S": 30e-6, "H2O": 20e-6}

In [54]:
results = traverse(
    temperature,
    pressure,
    concs,
    sample_length=10,
)

In [55]:
df_d = pd.DataFrame(results.initfinaldiff)
display(df_d)

,initial,final,change
SO2,10.0,10.0,2.044705e-08
NO2,50.0,50.0,-1.974935e-08
H2S,30.0,30.0,-2.474075e-08
H2O,20.0,20.0,8.210321e-12


In [56]:
### statistics
analyse = AnalyseSampling(results.data, markdown=True)

analyse.mean_sampling()
analyse.reaction_paths()
analyse.reaction_statistics()

mean = analyse.mean_data
paths = pd.DataFrame(analyse.common_paths)
stats = pd.DataFrame(analyse.stats)

df_m_t = pd.DataFrame(mean).T
df_m_t = df_m_t[df_m_t["value"] != 0]

df_m = pd.DataFrame(
    {
        "comps": list(df_m_t.T.keys()),
        "values": df_m_t["value"].values,
        "variance": df_m_t["variance"].values,
        "variance_minus": -df_m_t["variance"].values,
    }
)

In [57]:
import plotly.express as px

fig = px.bar(
    df_m,
    x="comps",
    y="values",
    error_y="variance",
    error_y_minus="variance_minus",
    labels={"comps": "", "values": "\u0394 ppm"},
    color="values",
    color_continuous_scale="tropic_r",
    hover_data={
        "values": False,
        "comps": False,
        "variance": False,
        "error": (":.2e", df_m["variance"]),
        "specie": df_m["comps"],
        "PPM": (":.1f", df_m["values"]),
    },
    width=1000,
    height=500,
)

fig.update_xaxes(showgrid=False, tickangle=-60, tickmode="linear")

In [58]:
display(paths)

,paths,k,frequency
0,2H<sub>2</sub>S + 3O<sub>2</sub> = 2H<sub>2</s...,7.65e+173 \n 3.62e-129,1


In [59]:
display(stats)

,index,k,frequency
0,2 NH3 + 2 SO2 = H2 + 2 H2S + 2 NO2,3.62e-129,3
1,2 NH3 + 3 SO2 = 3 H2S + 2 NO2 + O2,1.43e-172,2
2,2 H2S + 3 O2 = 2 H2O + 2 SO2,7.65e+173,2
3,3 H2 + SO2 = 2 H2O + H2S,4.69e+43,1
4,H2S + 3 H2SO4 = 4 H2O + 4 SO2,1.7e+43,1
5,24 H2S + 16 NO2 = 16 NH3 + S8 + 16 SO2,7.45e-200,1
